## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Read the csv data
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

### Checking Missing and Incorrect Values

In [4]:
#List of columns with NaN values
df.columns[df.isna().any()]

Index(['waterfront', 'view', 'yr_renovated'], dtype='object')

In [5]:
#Checking for Nan and non-numeric values:
print('Nan and non-numeric values summary:\n')

for column in df.columns:
    
    column_dtype = df.dtypes[column]
    nnv = pd.to_numeric(df[column], errors='coerce').isnull().sum()
    nan = df[column].isnull().sum()
    
    if nnv>0 or nan>0:
    
        print('Column name:\t\t\t' + column)
        print(f'Column data type:\t\t{column_dtype}')
        print(f'# of Nan values:\t\t{nan}')
        print(f'# of non-numeric values:\t{nnv-nan}\n') 
        
        #add proportion of missing data - proportion_of_nas = (sum(df[column].isnull())/len(df[column])) * 100 

Nan and non-numeric values summary:

Column name:			date
Column data type:		object
# of Nan values:		0
# of non-numeric values:	21597

Column name:			waterfront
Column data type:		float64
# of Nan values:		2376
# of non-numeric values:	0

Column name:			view
Column data type:		float64
# of Nan values:		63
# of non-numeric values:	0

Column name:			sqft_basement
Column data type:		object
# of Nan values:		0
# of non-numeric values:	454

Column name:			yr_renovated
Column data type:		float64
# of Nan values:		3842
# of non-numeric values:	0



In [6]:
#Checking correct values in yr_renovated column
df[df['yr_renovated']==0.0].shape[0]

17011

### Checking Dupliactes in 'id' Column

In [7]:
#Searching for duplicated in 'id' column
df[df.duplicated(subset='id')==True].shape[0]

177

In [8]:
df[df.duplicated()].any().any()

False

### Data Processing

In [9]:
#Creating a copy of df
df_processed = df.copy()

In [10]:
#Recalculate sq basement based on sqft_living and sqft_above
df_processed['sqft_basement'] = df_processed['sqft_living'] - df_processed['sqft_above']

In [11]:
#Binarize yr_renovated column renovated/not renovated
df_processed['is_renovated'] = (df_processed['yr_renovated'] > 0).astype(int)
df_processed['is_renovated']

0        0
1        1
2        0
3        0
4        0
        ..
21592    0
21593    0
21594    0
21595    0
21596    0
Name: is_renovated, Length: 21597, dtype: int64

In [12]:
#Dropping duplicates based on id column
df.drop_duplicates(subset='id', inplace=True)

In [13]:
# Changing NaN values to 0 in waterfront
pd.to_numeric(df_processed['waterfront'], downcast='signed', errors='coerce')
df_processed['waterfront'] = df_processed['waterfront'].fillna(0).astype(int)
# Changing NaN values to 2 in view
df_processed['view'] = df_processed['view'].fillna(2).astype(int)

In [14]:
df_processed['view'].value_counts()

0    19422
2     1020
3      508
1      330
4      317
Name: view, dtype: int64

In [15]:
#Date to data format
df_processed['date'] = pd.to_datetime(df_processed.date)
df_processed['date'] = df_processed['date'].dt.year

In [16]:
#Resetting index
df_processed.reset_index(drop=True, inplace=True)

In [17]:
#Dropping id column
df_processed.drop(columns='id', inplace=True)

In [18]:
df_processed.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,is_renovated
0,2014,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,1180,0,1955,0.0,98178,47.5112,-122.257,1340,5650,0
1,2014,538000.0,3,2.25,2570,7242,2.0,0,0,3,...,2170,400,1951,1991.0,98125,47.7210,-122.319,1690,7639,1
2,2015,180000.0,2,1.00,770,10000,1.0,0,0,3,...,770,0,1933,NaN,98028,47.7379,-122.233,2720,8062,0
3,2014,604000.0,4,3.00,1960,5000,1.0,0,0,5,...,1050,910,1965,0.0,98136,47.5208,-122.393,1360,5000,0
4,2015,510000.0,3,2.00,1680,8080,1.0,0,0,3,...,1680,0,1987,0.0,98074,47.6168,-122.045,1800,7503,0


### Saving df_processed as a pickle file

In [77]:
with open('df_processed.pickle', 'wb') as f:
    pickle.dump(df_processed, f, pickle.HIGHEST_PROTOCOL)